In [2]:
# Set global utool flags
import utool as ut
ut.util_io.__PRINT_WRITES__ = False
ut.util_io.__PRINT_READS__ = False
ut.util_parallel.__FORCE_SERIAL__ = True
ut.util_cache.VERBOSE_CACHE = False
ut.NOT_QUIET = False

# Matplotlib stuff
%pylab inline
#%load_ext autoreload
#%autoreload

import ibeis
#ibeis.experiments.experiment_harness.USE_BIG_TEST_CACHE = False
#ibeis.init.filter_annots.USE_ACFG_CACHE = False

# Define database for this test run
db = 'PZ_Master1'
#db = 'GZ_ALL'
#db = 'NNP_MasterGIRM_core'
 
# Setup database specific configs
if db == 'PZ_MTEST' or db == 'GZ_ALL':
    seperability_annot_cfgs = ['ctrl']
else:
    seperability_annot_cfgs = ['timequalctrl']

varysize = ['varysize']
varypername = ['varypername']

ALL_TESTS = False
SHOW_DBINFO = False

`%matplotlib` prevents importing * from pylab and numpy
/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:1318: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Populating the interactive namespace from numpy and matplotlib


# Database Information

In [ ]:
if SHOW_DBINFO:
    _ = ibeis.other.dbinfo.get_dbinfo(ibs)
    print('Command Line:\n    python -m ibeis.other.dbinfo --exec-get_dbinfo:0 --db {db}'.format(db=db))

if SHOW_DBINFO:
    import ibeis
    ibs = ibeis.opendb(db=db)
    ibeis.other.dbinfo.show_image_time_distributions(ibs, ibs.get_valid_gids())

In [ ]:
if SHOW_DBINFO:
    latex_stats = ibeis.other.dbinfo.latex_dbstats([ibs], table_position='[h]') + '\n%--'
    #print(latex_stats)}
    pdf_fpath = ut.compile_latex_text(latex_stats, dpath=None, verbose=False, quiet=True, pad_stdout=False)
    pdf_fpath = ut.tail(pdf_fpath, n=2)
    print(pdf_fpath)
    from IPython.display import HTML
    HTML('<iframe src="%s" width=700 height=350></iframe>' % pdf_fpath)

# Baseline Experiments

In [ ]:
test_result = ibeis.run_experiment(
    e='rank_cdf', 
    db=db, 
    a=['unctrl', 'ctrl::unctrl_comp'],
    t=['baseline'])
#test_result.print_acfg_info()
_ = test_result.draw_func()

In [ ]:
test_result = ibeis.run_experiment(
    e='timedelta_pie',
    db=db, 
    a=['ctrl::unctrl_comp'], 
    t=['baseline'],
    truepos=True)
test_result.draw_func()

test_result = ibeis.run_experiment(
    e='timedelta_pie',
    db=db, 
    a=['ctrl::unctrl_comp'], 
    t=['baseline'],
    falsepos=True)
test_result.draw_func()

## Time Experiment

In [ ]:
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['timectrl', 'ctrl'], 
    t=['baseline'])
#test_result.print_acfg_info()
test_result.draw_func()

test_result = ibeis.run_experiment(
    e='timedelta_pie',
    db=db, 
    a=['timectrl'], 
    t=['baseline'],
    falsepos=True, truepos=True)
test_result.draw_func()

# Database Size Experiments

In [ ]:
if ALL_TESTS:
    test_result = ibeis.run_experiment(
        e='rank_surface',
        db=db, 
        a=['varysize'], 
        t=['candk'])
    #test_result.print_unique_annot_config_stats()
    #test_result.print_acfg_info()
    test_result.draw_func()
if True:
    test_result = ibeis.run_experiment(
        e='rank_surface',
        db=db, 
        a=['varysize_td'], 
        t=['candk'])
    #test_result.print_unique_annot_config_stats()
    #test_result.print_acfg_info()
    test_result.draw_func()
    
if True:
    # This test requires a little bit of relaxation to get enough data
    test_result = ibeis.run_experiment(
        e='rank_surface',
        db=db, 
        a=['varysize_tdqual:qmin_pername=3,dpername=[1,2]'], 
        t=['candk'])
    #test_result.print_unique_annot_config_stats()
    #test_result.print_acfg_info()
    test_result.draw_func()

# Effect of K (explicit test for larger databases)

### Test of K

In [ ]:
TEST_K = False
if TEST_K:
    if ALL_TESTS:
        test_result = ibeis.run_experiment(
            e='rank_cdf',
            db=db, 
            a=['ctrl'], 
            t=['candk'], lazy=True)
        test_result = test_result()
        test_result.draw_func()

    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['timectrl'], 
        t=['candk'])
    test_result.draw_func()

    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['timequalctrl'], 
        t=['candk'])
    #test_result.print_acfg_info(per_qual=True)
    test_result.draw_func()

### Test of Knorm

In [ ]:
TEST_KNORM = False

if TEST_KNORM:
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['ctrl'], 
        t=['candknorm'])
    #test_result.print_acfg_info(per_qual=True)
    test_result.draw_func()

    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['timectrl'], 
        t=['candknorm'])
    #test_result.print_acfg_info(per_qual=True)
    test_result.draw_func()

    # Test Knorm 
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['timequalctrl'], 
        t=['candknorm'])
    #test_result.print_acfg_info(per_qual=True)
    test_result.draw_func()

### Joint Test that controls for database size to compare between the timequalctrl and timectrl 

In [ ]:
if TEST_KNORM:
    # 10 being good in timectrl seems to result from database size?
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['timequalctrl::timectrl'], 
        #t=['candknorm:K=[2,3,4,10]']
        t=['candknorm:K=[2,3,4]']
        )
    #test_result.print_acfg_info(per_qual=True)
    test_result.draw_func()

# Scoring Mechanism Experiments

In [ ]:
# OLD Namescore Experiment
if 0:
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        #a=varypername, 
        a=['varypername'],
        t=['candscoremech'])
    test_result.draw_func()

test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['varypername_td'],
    t=['candscoremech'])
test_result.draw_func()

if 0:
    # Not enough data to run this (can hack off exclude_reference to get something)
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['varypername_tdqual:exclude_reference=False,qmin_pername=3'],
        t=['candscoremech'])
    test_result.draw_func()

In [ ]:
# Test significance of adding a single picture
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    #a=['timectrl:qmin_pername=3,dpername=2'],
    a=['varypername2_td'],
    #t=['candscoremech']
    t=['invarbest'],
)
test_result.draw_func()

# Viewpoint Experiments

In [ ]:
try:
    ibeis.rrr()
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['viewdiff'], 
        t=['baseline'])
    
    test_result.draw_func()
    
    
except AssertionError as ex:
    ut.printex(ex, 'Database does not support this test')

In [ ]:
test_result.draw_cases(f=[':index=3:4'], annot_modes=[1])

In [ ]:
try:
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['viewdiff'], 
        t=['invarbest:K=1'])
    test_result.draw_func()
except AssertionError as ex:
    ut.printex(ex, 'Database does not support this test')

In [3]:
try:
    ibeis.rrr()
    # Hardly get anything when timedelta is small
    # Not enough data for this. 
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['viewdiff:min_timedelta=30m,require_timestamp=True,dhack_extra=True'], 
        t=['invarbest:K=1'])
    #test_result.draw_func()

except AssertionError as ex:
    ut.printex(ex, 'Database does not support this test')

RELOAD: [ibeis] __name__=ibeis
Equivalent Command Line:
ibeis -e rank_cdf --db PZ_Master1 -a viewdiff:min_timedelta=30m,require_timestamp=True,dhack_extra=True -t invarbest:K=1 --show

[ibs.__init__] new IBEISController
[ibs._init_dirs] ibs.dbdir = '/media/raid/work/PZ_Master1'
[ibs.__init__] END new IBEISController

testnameid="PZ_Master1 ['invarbest:K=1']['viewdiff:min_timedelta=30m,require_timestamp=True,dhack_extra=True']"
annot_cfgstr = _QSUUIDS((7)8b4kdc5ai77+kuvn)_DSUUIDS((530)g2xhk5wbj5pxczuj)
pipe_cfgstr= _vsmany_NN(single,K1+1,padk=False,last,cks800)_NNWeight(lnbnn)_SV(0.01;2.0;1.57minIn=4,nNRR=50,nARR=6,nsum,cdl,+wi,+fc,)_AGG(nsum)_FLANN(8_kdtrees)_RRVsOne(False)_FEATWEIGHT(OFF)_FEAT(hesaff+sift_affine_invariance=False)_CHIP(sz450)_HACK(augment_queryside)
pipe_hashstr = mgkzkgpydxuzqnti
[PZ_Master1 cfg 0/2][add_chip_feat] adding 28 / 530 new feat for config_rowid=19
[PZ_Master1 cfg 0/2][add_chip_feat] adding to sql in chunks with chunksize=128
[util_parallel.generate] genera

In [ ]:
_ = test_result.draw_cases(
    annot_modes=[0, 1],
    f=[':index=0:8,orderby=gtscore'],  # Orderby means less when multi configs used
    )()

There is not enough data in the plains zebra dataset to make this test workable. The results are very poor. 

In [ ]:
if False:
    try:
        # Look at interaction between viewpoint and K
        # There really isn't anything significant
        test_result = ibeis.run_experiment(
            e='rank_cdf',
            db=db, 
            a=['viewdiff'], 
            t=['baseline:K=[1,2,4,7]'])
        test_result.draw_func()
    except AssertionError as ex:
        ut.printex(ex, 'Database does not support this test')

# Invariance Experiments

In [ ]:
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['ctrl'], 
    t=['invar'])
test_result.draw_func()

# Invariance + Viewpoint Experiment
try:
    test_result = ibeis.run_experiment(
        e='rank_cdf',
        db=db, 
        a=['viewdiff'], 
        t=['invar'], 
        test_cfgx_slice=slice(6, None))
    test_result.draw_func()
except AssertionError as ex:
    ut.printex(ex, 'Database does not support this test')
    
    
# Invariance + Time Experiment
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['timectrl'], 
    t=['invar4'])
_ = test_result.draw_func()

# Invariance + Quality + Time Experiment
test_result = ibeis.run_experiment(
    e='rank_cdf',
    db=db, 
    a=['timequalctrl'], 
    t=['invar4'])
_ = test_result.draw_func()

Notice how AQH=On, AI=On does much better as the ranks increase for the viewpoint test. This suggests that affine invariance is important for matching disparage viewpoints, however it seems that being paied with a little bit of orientation invariance is very helpful.

# Score Separability Experiments

## Scores of Success Cases

In [ ]:
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=False,min_gf_timedelta=None'],
)
_ = test_result.draw_func()

## Scores of All Cases

In [ ]:
# Score Seperability with NO FILTERING
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=None,min_gf_timedelta=None']
)
_ = test_result.draw_func()

In [ ]:
test_result = ibeis.run_experiment(
    e='taghist',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=None,min_gf_timedelta=None']
)
_ = test_result.draw_func()

## Scores of Success Cases  (with large timedelta groundfalse)

In [ ]:
# Score Seperability of Good Cases where large timedelta gf are gone
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=False,min_gf_timedelta=12h']
)
_ = test_result.draw_func()

## Scores of All Cases (with large timedelta groundfalse) 

In [ ]:
# Score Seperability of 
test_result = ibeis.run_experiment(
    e='scores',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=None,min_gf_timedelta=12h']
) 
_ = test_result.draw_func()

In [ ]:
# Investigate the reasons for the high scoring groundfalse cases
test_result = ibeis.run_experiment(
    e='taghist',
    db=db, 
    a=['timecontrolled'], 
    t=['invarbest'],
    f=[':fail=None,min_gf_timedelta=12h']
)
_ = test_result.draw_func()

# Tags of all failure cases

In [ ]:
test_result = ibeis.run_experiment(
    e='taghist',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=True,min_gf_timedelta=None']
)
_ = test_result.draw_func()

# Tags of failure cases (largetimedelta groundfalse)

In [ ]:
test_result = ibeis.run_experiment(
    e='taghist',
    db=db, 
    a=seperability_annot_cfgs, 
    t=['invarbest'],
    f=[':fail=True,min_gf_timedelta=12h']
)
_ = test_result.draw_func()

------------------

# Individual Results 
* Higher scores are on the Left
* Groundtrue matches have a green border

In [ ]:
t = ['invarbest']
a = seperability_annot_cfgs
draw_case_kw = dict(show_in_notebook=True, annot_modes=[0, 1])

### Highest Scoring Success Cases

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=False,min_gf_timedelta=12h,index=0:2,orderby=gtscore,reverse=True,without_gf_tag=Photobomb'],
    **draw_case_kw)
_ = test_result.draw_func()

### Failure Cases (sorted by groundtrue score)

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=True,min_gf_timedelta=12h,index=0:3,orderby=gtscore,reverse=True'],
    **ut.augdict(draw_case_kw, annot_modes=[1]))
_ = test_result.draw_func()

### Failure Cases (sorted by groundfalse score) 

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=True,min_gf_timedelta=12h,index=0:3,orderby=gfscore,reverse=True'],
    **draw_case_kw)
_ = test_result.draw_func()

### Failure Cases (sorted by groundfalse score and no timedelta restriction) 

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':fail=True,index=0:2,orderby=gfscore,reverse=True'],
    **draw_case_kw)
_ = test_result.draw_func()

## Interesting Tags

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    #t=['default', 'invarbest'],
    f=[':index=0:5,with_tag=interesting'],
    **draw_case_kw)
_ = test_result.draw_func()

#### Specific Interesting Tag (varying pipecfg)

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    #t=t,
    t=['default', 'invarbest', 'invarbest:SV=False'],
    qaid_override=[2604],
    **draw_case_kw)
_ = test_result.draw_func()

## Occlusion Tags

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':index=0:10,with_tag=occlusion,without_gf_tag=Photobomb,without_gt_tag=viewpoint'],
    **draw_case_kw)
_ = test_result.draw_func()

### Photobomb Cases

In [ ]:
test_result = ibeis.run_experiment(
    e='draw_cases',
    db=db, 
    a=a, 
    t=t,
    f=[':index=0:7:2,with_tag=Photobomb'],
    **draw_case_kw)
_ = test_result.draw_func()

It still seems unacceptable that even a correctly matched animal should have a score less than 1. 
